Outstanding issues:

    - memory usage when handling the data, occurs with gdal and sometimes with rasterio too
        pushes to basically the maxmimum it can use, and still takes a long time
        see below code with getting averages, there wasn't enough memory
        tends to drain battery too

In [11]:
conda --version

conda 4.10.3
Note: you may need to restart the kernel to use updated packages.



In [1]:
import sys
print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]


In [14]:
import rasterio
import pandas as pd
import numpy as np
import zipfile
from io import BytesIO

def tiff_to_csv(tiff_path):
    # Open the TIFF file
    with rasterio.open(tiff_path) as src:
        # Read the data
        data = src.read(1)  # Assuming single band, use src.read() for all bands
        
        # Get coordinates
        height, width = data.shape
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        xs, ys = rasterio.transform.xy(src.transform, rows, cols)
        
        # Convert xs and ys to NumPy arrays if they're not already
        xs = np.array(xs)
        ys = np.array(ys)
        
        # Create a dataframe
        df = pd.DataFrame({
            'x': xs.flatten(),
            'y': ys.flatten(),
            'value': data.flatten()
        })
    return df

def tiff_to_csv_value_by_year(tiff_path, year):
    """
    Read a multi-band TIFF file and output the value for a specific year at each XY location.
    
    :param tiff_path: Path to the TIFF file
    :param year: The specific year to get the value for
    :return: DataFrame with x, y, and value for the specified year
    """
    with rasterio.open(tiff_path) as src:
        # Get the number of bands
        num_bands = src.count
        
        # Assume the first band is the earliest year
        start_year = src.tags().get('YEAR', str(year - num_bands + 1))
        start_year = int(start_year)
        
        # Calculate which band corresponds to the requested year
        year_index = year - start_year
        
        if year_index < 0 or year_index >= num_bands:
            raise ValueError(f"Year {year} is out of range. Data ranges from {start_year} to {start_year + num_bands - 1}.")
        
        # Read the data for the specified year
        data = src.read(year_index + 1)  # rasterio uses 1-based indexing for bands
        
        # Get coordinates
        height, width = data.shape
        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        xs, ys = rasterio.transform.xy(src.transform, rows, cols)
        
        # Convert xs and ys to NumPy arrays
        xs = np.array(xs)
        ys = np.array(ys)
        
        # Create a dataframe
        df = pd.DataFrame({
            'x': xs.flatten(),
            'y': ys.flatten(),
            f'value_{year}': data.flatten()
        })
        
        # Group by x and y coordinates and calculate the mean
        df_mean = df.groupby(['x', 'y'])[f'value_{year}'].mean().reset_index()
        df_mean = df_mean.rename(columns={f'value_{year}': f'mean_value_{year}'})
        
    return df_mean

def tiff_to_csv_from_zip(zip_path, tiff_name):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        with zip_ref.open(tiff_name) as tiff_file:
            with rasterio.open(BytesIO(tiff_file.read())) as src:
                # Read the data
                data = src.read(1)  # Assuming single band, use src.read() for all bands
                
                # Get coordinates
                height, width = data.shape
                cols, rows = np.meshgrid(np.arange(width), np.arange(height))
                xs, ys = rasterio.transform.xy(src.transform, rows, cols)
                
                # Convert xs and ys to NumPy arrays if they're not already
                xs = np.array(xs)
                ys = np.array(ys)
                
                # Create a dataframe
                df = pd.DataFrame({
                    'x': xs.flatten(),
                    'y': ys.flatten(),
                    'value': data.flatten()
                })
    return df

# Example usage
# tiff_to_csv_from_zip('data.zip', 'input.tif', 'output.csv')

In [3]:
# csv_path='home\Documents\classes\biomassProject\ClusterAnalysis'
# tiff_path='swbiomass_data1\gedi\gedi_2019-2023.tif'
# zip_path='D:\Biomass\swbiomass_data1.zip'

# data=tiff_to_csv_from_zip(zip_path, tiff_path)
# data

In [28]:
import os
import zipfile

path_to_folder = r"D:\Biomass\swbiomass_data1.zip"

if os.path.exists(path_to_folder):
    print("Files in the zip archive:")
    with zipfile.ZipFile(path_to_folder, 'r') as zip_ref:
        for file in zip_ref.namelist():
            print(file)
else:
    print("The specified path does not exist.")

Files in the zip archive:
swbiomass_data1/chopping/
swbiomass_data1/chopping/chopping_2000-2021.tif
swbiomass_data1/chopping/chopping_2000-2021.tif.aux.json
swbiomass_data1/esa_cci/
swbiomass_data1/esa_cci/esa.cci_N40W110_2010.2017-2020.tif
swbiomass_data1/esa_cci/esa.cci_N40W110_2010.2017-2020.tif.aux.json
swbiomass_data1/esa_cci/esa.cci_N40W120_2010.2017-2020.tif
swbiomass_data1/esa_cci/esa.cci_N40W120_2010.2017-2020.tif.aux.json
swbiomass_data1/esa_cci/esa.cci_N40W130_2010.2017-2020.tif
swbiomass_data1/esa_cci/esa.cci_N40W130_2010.2017-2020.tif.aux.json
swbiomass_data1/esa_cci/esa.cci_N50W120_2010.2017-2020.tif
swbiomass_data1/esa_cci/esa.cci_N50W120_2010.2017-2020.tif.aux.json
swbiomass_data1/esa_cci/esa.cci_N50W130_2010.2017-2020.tif
swbiomass_data1/esa_cci/esa.cci_N50W130_2010.2017-2020.tif.aux.json
swbiomass_data1/gedi/
swbiomass_data1/gedi/gedi_2019-2023.tif
swbiomass_data1/gedi/gedi_2019-2023.tif.aux.json
swbiomass_data1/liu/
swbiomass_data1/liu/liu_1993-2012.tif
swbiomass_dat

In [59]:
import os
print("Current working directory:", os.getcwd())

# On Windows, list available drives
if os.name == 'nt':
    import win32api
    drives = win32api.GetLogicalDriveStrings()
    print("Available drives:", drives.split('\000')[:-1])

# On Unix-like systems (macOS, Linux), list contents of /Volumes or /media
if os.name == 'posix':
    print("Contents of /Volumes:", os.listdir('/Volumes'))
    # or
    print("Contents of /media:", os.listdir('/media'))

Current working directory: C:\Users\aizax\Documents\classes\biomassProject\ClusterAnalysis
Available drives: ['C:\\', 'D:\\']


In [57]:
tiff_path=r'D:\Biomass\swbiomass_data1\swbiomass_data1\esa_cci\esa.cci_N40W110_2010.2017-2020.tif'

data=tiff_to_csv(tiff_path)
data

,x,y,value
0,-109.999556,39.999556,0
1,-109.998667,39.999556,0
2,-109.997778,39.999556,0
3,-109.996889,39.999556,0
4,-109.996000,39.999556,0
...,...,...,...
126562495,-100.004000,30.000444,65
126562496,-100.003111,30.000444,50
126562497,-100.002222,30.000444,50
126562498,-100.001333,30.000444,31


In [15]:
tiff_path=r'D:\Biomass\swbiomass_data1\swbiomass_data1\esa_cci\esa.cci_N40W110_2010.2017-2020.tif'

data=tiff_to_csv_value_by_year(tiff_path,2010)
data

MemoryError: Unable to allocate 1.89 GiB for an array with shape (2, 126562500) and data type float64

In [ ]:
#Save to CSV
csv_path='home\Documents\classes\biomassProject\ClusterAnalysis\esa.csv'

df.to_csv(csv_path, index=False)

# Using Gdal

In [2]:
#FROM YANG

import numpy as np
import pandas as pd
from osgeo import gdal

rasterfn = r"D:\Biomass\swbiomass_data1\swbiomass_data1\esa_cci\esa.cci_N40W110_2010.2017-2020.tif"
outf = r"D:\Biomass\average_esa.csv"
raster = gdal.Open(rasterfn) # load tiff file

RasterCount = raster.RasterCount # get band number

year_list = [] # init year list
value_list = [] # init value list
for i in range(1,RasterCount+1): # read each band
    band = raster.GetRasterBand(i) # get band
    band_name = band.GetDescription() # get band name

    array = band.ReadAsArray() # read as array
    array = np.array(array,dtype=float) # convert array to float
    array[array<=0] = np.nan # for AGB, set 0 to nan
    array_mean = np.nanmean(array) # calculate mean, ignore nan

    year_list.append(band_name) # append year
    value_list.append(array_mean) # append value
df = pd.DataFrame({'year':year_list,'value':value_list}) # create dataframe
df.to_csv(outf,index=False) # save dataframe to csv

C:\Users\aizax\anaconda3\envs\py310\lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [3]:
df

,year,value
0,2010,33.212983
1,2017,37.197625
2,2018,37.171370
3,2019,36.110136
4,2020,38.535405


In [5]:
#attempt to use gdal to get data by XY location for given year

# Replace with the path to your raster file
rasterfn = r"D:\Biomass\swbiomass_data1\swbiomass_data1\esa_cci\esa.cci_N40W110_2010.2017-2020.tif"
# Replace with the year you're interested in
target_year = 2010

raster = gdal.Open(rasterfn)
RasterCount = raster.RasterCount

target_band = None
for i in range(1, RasterCount + 1):
    band = raster.GetRasterBand(i)
    band_name = band.GetDescription()
    if str(target_year) in band_name:
        target_band = band
        break

if target_band is None:
    print(f"No band found for year {target_year}")
else:
    # Get raster dimensions
    width = raster.RasterXSize
    height = raster.RasterYSize
    
    # Read the entire band as a numpy array
    data = target_band.ReadAsArray(0, 0, width, height)
    
    # Create a 2D array to store averaged values
    averaged_values = np.zeros((height, width))
    
    # Calculate average for each x,y location
    for y in range(height):
        for x in range(width):
            averaged_values[y, x] = np.mean(data[y, x])
    
    print(f"Averaged values for year {target_year}:")
    print(averaged_values)

# Close the dataset
raster = None

Averaged values for year 2010:
[[ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 [ 0.  0.  0. ...  0.  0.  0.]
 ...
 [ 0.  0.  6. ...  3.  2. 15.]
 [ 0.  0.  0. ...  8.  3.  1.]
 [ 0.  0.  0. ... 50. 31.  0.]]
